In [2]:
import os

import polars as pl

from dotenv import load_dotenv

from eco_stats import BLSClient

load_dotenv()

True

In [3]:
bls = BLSClient(api_key=os.getenv('BLS_API_KEY'))

In [4]:
ces_1 = (
    bls.get_series(
        series_ids=['CES0500000001', 'CEU0500000001'],
        start_year='2015',
        end_year='2025'
    )
    .select(
        adjusted=pl.col('series_id').str.slice(2, 1).eq('S'),
        ref_date=pl.col('date'),
        value=pl.col('value'),
    )
    .sort('ref_date')
    .pivot(
        'adjusted',
        index='ref_date',
        values='value'
    )
    .select(
        ref_date=pl.col('ref_date'),
        ces_nsa=pl.col('false').mul(1000),
        ces_sa=pl.col('true').mul(1000),
        
    )
    .with_columns(
        ces_nsa_diff=pl.col('ces_nsa').sub(pl.col('ces_nsa').shift(1)),
        ces_sa_diff=pl.col('ces_sa').sub(pl.col('ces_sa').shift(1))
    )
    .with_columns(
        ces_nsa_growth=pl.col('ces_nsa_diff').truediv(pl.col('ces_nsa').shift(1)),
        ces_sa_growth=pl.col('ces_sa_diff').truediv(pl.col('ces_sa').shift(1)),
    )
    .filter(
        pl.col('ref_date').ge(pl.date(2016, 1, 12))
    )
    .with_columns(
        ces_nsa_index=pl.col('ces_nsa_growth').add(1).cum_prod(),
        ces_sa_index=pl.col('ces_sa_growth').add(1).cum_prod()
    )
    .select(
        ref_date=pl.col('ref_date'),
        ces_nsa_level=pl.col('ces_nsa'),
        ces_nsa_diff=pl.col('ces_nsa_diff'),
        ces_nsa_growth=pl.col('ces_nsa_growth'),
        ces_nsa_index=pl.col('ces_nsa_index'),
        ces_sa_level=pl.col('ces_sa'),
        ces_sa_diff=pl.col('ces_sa_diff'),
        ces_sa_growth=pl.col('ces_sa_growth'),
        ces_sa_index=pl.col('ces_sa_index'),
    )
)

rebase_nsa = (
    ces_1
    .filter(pl.col('ref_date').eq(pl.date(2020, 3, 12)))
    .get_column('ces_nsa_index')
    .item()
)

rebase_sa = (
    ces_1
    .filter(pl.col('ref_date').eq(pl.date(2020, 3, 12)))
    .get_column('ces_sa_index')
    .item()
)

ces = (
    ces_1
    .with_columns(
        ces_nsa_index=pl.col('ces_nsa_index').truediv(rebase_nsa).mul(100),
        ces_sa_index=pl.col('ces_sa_index').truediv(rebase_sa).mul(100)
    )
)

(
    ces
    .write_csv(
        '~/Projects/pp/data/ces_index.csv'
    )
)

In [5]:
qcew_1 = (
    bls.get_series(
        series_ids=['ENUUS00010510'],
        start_year='2015',
        end_year='2025'
    )
    .select(
        ref_date=pl.col('date'),
        qcew_nsa=pl.col('value'),
    )
    .sort('ref_date')
    .with_columns(
        qcew_nsa_diff=pl.col('qcew_nsa').sub(pl.col('qcew_nsa').shift(1))
    )
    .with_columns(
        qcew_nsa_growth=pl.col('qcew_nsa_diff').truediv(pl.col('qcew_nsa').shift(1)),
    )
    .with_columns(
        qcew_nsa_index=pl.col('qcew_nsa_growth').add(1).cum_prod()
    )
    .filter(
        pl.col('ref_date').ge(pl.date(2016, 1, 12))
    )
    .select(
        ref_date=pl.col('ref_date'),
        qcew_nsa_level=pl.col('qcew_nsa'),
        qcew_nsa_diff=pl.col('qcew_nsa_diff'),
        qcew_nsa_growth=pl.col('qcew_nsa_growth'),
        qcew_nsa_index=pl.col('qcew_nsa_index'),
    )
)

rebase_nsa = (
    qcew_1
    .filter(pl.col('ref_date').eq(pl.date(2020, 3, 12)))
    .get_column('qcew_nsa_index')
    .item()
)

qcew = (
    qcew_1
    .with_columns(
        qcew_nsa_index=pl.col('qcew_nsa_index').truediv(rebase_nsa).mul(100)
    )
)

(
    qcew
    .write_csv(
        '~/Projects/pp/data/qcew_index.csv'
    )
)